In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from scipy.optimize import curve_fit
import math
os.chdir('C:\\Users\\dziubins\\Home\\Research\\Energy Loss Optical Scintillation System\\Data\\')

#-------------Read in Photon File---------------#
col_names = list(range(1,121,1))
#folder = 'eloss_r8520_m\\eloss_r8520_m\\build\\'
phodf = pd.read_table("PMT Sensitivity with G4\\r8520_59Ni.photon", sep=' ', names=col_names, index_col=False )

#-------------Read in PMT Calibration File---------------#
caldf = pd.read_excel("UV PMT Calibration - Version 6.xlsx", sheet_name=[0,1], usecols="B:R" )

#-------------Cleanup PMT Calibration File---------------#
led1_short = caldf[0].iloc[::3][:-1]
led2_short = caldf[0].iloc[1::3]
led3_short = caldf[0].iloc[2::3]
led1_long = caldf[1].iloc[::3][:-1]
led2_long = caldf[1].iloc[1::3]
led3_long = caldf[1].iloc[2::3]
#print(len(led3_short),len(led3_long) )

led1 = pd.concat([led1_short, led1_long], ignore_index=True)
led2 = pd.concat([led2_short, led2_long], ignore_index=True)
led3 = pd.concat([led3_short, led3_long], ignore_index=True)
led2['PMT#'], led3['PMT#'] = led1['PMT#'], led1['PMT#']
led1 = led1.sort_values(by=['PMT#']).reset_index()
led2 = led2.sort_values(by=['PMT#']).reset_index()
led3 = led3.sort_values(by=['PMT#']).reset_index()
led3 = led3.drop(['index','Location','Warm Up Time'], axis=1)



clean_led1 = [x for x in list(led1['Calibrated']) if isinstance(x, (int, float)) and math.isnan(x)==False]
clean_led2 = [x for x in list(led2['Calibrated']) if isinstance(x, (int, float)) and math.isnan(x)==False]
clean_led3 = [x for x in list(led3['Calibrated']) if isinstance(x, (int, float)) and math.isnan(x)==False]

#-------------Read in PMT Mapping File---------------#
mapdf = pd.read_excel("ELOSS Channel Mapping.xlsx", sheet_name=[0], usecols="J:K", skiprows=[0,1], nrows=120)
mapdf = mapdf[0].sort_values(by=['PMT Serial '])
mapdf = mapdf.reset_index()
mapdf.drop('index', axis=1);

In [3]:
phodf

,1,2,3,4,5,6,7,8,9,10,...,111,112,113,114,115,116,117,118,119,120
0,624,626,479,410,300,253,180,151,114,76,...,55,113,238,721,5207,15705,1688,323,123,74
1,916,792,642,467,361,261,202,133,100,95,...,192,352,683,1581,3237,1711,17055,1541,335,137
2,23,29,46,50,96,172,399,1105,4045,5580,...,343,260,168,134,108,186,190,235,225,234
3,798,938,886,732,564,399,270,191,152,113,...,469,932,1995,3148,2436,1759,2762,2217,1097,562
4,480,567,648,633,570,457,345,271,200,153,...,409,1656,12107,2234,502,1389,1276,921,596,371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,48,54,83,152,287,597,1632,4325,3453,1232,...,510,370,277,194,157,231,240,271,275,315
2996,681,3073,9537,1765,435,165,83,43,32,20,...,494,575,671,702,630,314,530,840,1370,1975
2997,324,464,636,752,775,693,566,450,343,241,...,3309,3934,1590,599,292,735,726,706,568,412
2998,95,150,279,589,1301,2730,3489,1883,837,368,...,806,657,501,367,250,334,395,417,474,477


In [9]:
# Make sure calibration data and channel mapping match! 

#for i in range(len(mapdf)):
#    print(mapdf.loc[i,'PMT Serial '],led3.loc[i,'PMT#'])

# Add PMT Locations to led3 (calibration data)
led3['PMT Location'] = mapdf['PMT Location']

# Get anode sensitivities for final calibration
anode_sens = np.ones(120)
for i in range(len(led3['PMT#'])):
    if isinstance(led3['Calibrated 2'][i], (int, float)) and math.isnan(led3['Calibrated 2'][i])==False:
        anode_sens[i] = led3['Calibrated 2'][i]
    elif isinstance(led3['Calibrated'][i], (int, float)) and math.isnan(led3['Calibrated'][i])==False:
        anode_sens[i] = led3['Calibrated'][i]
    else:
        anode_sens[i] = 45000
        
# Get anode sensitivities after course gain correction
#anode_sens = np.ones(120)
#for i in range(len(led3['PMT#'])):
#    if isinstance(led3['800 V'][i], (int, float)):
#        anode_sens[i] = led3['800 V'][i]
#    elif isinstance(led3['800 V'][i], str):
#        anode_sens[i] = 65600
#    else:
#        anode_sens[i] = np.mean(anode_sens)

# Normalize anode sensitivities to the average        
rel_anode_sens = anode_sens/np.mean(anode_sens)
    # get relative anode sensitivities
#rel_anode_sens[2] = 1

In [10]:
# Simplify dataframe to just PMT#, Location and Relative Sensitivity
simple_df = led3.loc[:,['PMT#','PMT Location']]
simple_df['Relative Anode Sensitivity'] = rel_anode_sens
sorted_df = simple_df.sort_values(by=['PMT Location']).reset_index().drop('index', axis=1)
sorted_df

,PMT#,PMT Location,Relative Anode Sensitivity
0,LV2050,S1-B1,0.995416
1,LV2029,S1-B2,1.006722
2,LV2060,S1-B3,0.997219
3,LV2148,S1-B4,0.998443
4,LV2125,S1-B5,1.001803
...,...,...,...
115,LV2199,S4-T1,0.996729
116,LV2084,S4-T2,0.998777
117,LV2169,S4-T3,0.997330
118,LV2155,S4-T4,0.999667


In [11]:
S1B = sorted_df.loc[:4]
S1L = sorted_df.loc[5:14].reset_index().drop('index', axis=1)
S1L10 = S1L.loc[1] 
S1L = pd.concat([S1L.drop(1), pd.DataFrame(S1L10).T], axis=0)
S1R = sorted_df.loc[15:24].reset_index().drop('index', axis=1)
S1R10 = S1R.loc[1]
S1R = pd.concat([S1R.drop(1), pd.DataFrame(S1R10).T], axis=0)
S1T = sorted_df.loc[25:29]


S2B = sorted_df.loc[30:34]
S2L = sorted_df.loc[(30+5):(30+14)].reset_index().drop('index', axis=1)
S2L10 = S2L.loc[1]
S2L = pd.concat([S2L.drop(1), pd.DataFrame(S2L10).T], axis=0)
S2R = sorted_df.loc[(30+15):(30+24)].reset_index().drop('index', axis=1)
S2R10 = S2R.loc[1]
S2R = pd.concat([S2R.drop(1), pd.DataFrame(S2R10).T], axis=0)
S2T = sorted_df.loc[(30+25):(30+29)]

S3B = sorted_df.loc[60:64]
S3L = sorted_df.loc[(60+5):(60+14)].reset_index().drop('index', axis=1)
S3L10 = S3L.loc[1]
S3L = pd.concat([S3L.drop(1), pd.DataFrame(S3L10).T], axis=0)
S3R = sorted_df.loc[(60+15):(60+24)].reset_index().drop('index', axis=1)
S3R10 = S3R.loc[1]
S3R = pd.concat([S3R.drop(1), pd.DataFrame(S3R10).T], axis=0)
S3T = sorted_df.loc[(60+25):(60+29)]

S4B = sorted_df.loc[90:94]
S4L = sorted_df.loc[(90+5):(90+14)].reset_index().drop('index', axis=1)
S4L10 = S4L.loc[1]
S4L = pd.concat([S4L.drop(1), pd.DataFrame(S4L10).T], axis=0)
S4R = sorted_df.loc[(90+15):(90+24)].reset_index().drop('index', axis=1)
S4R10 = S4R.loc[1]
S4R = pd.concat([S4R.drop(1), pd.DataFrame(S4R10).T], axis=0)
S4T = sorted_df.loc[(90+25):(90+29)]

g4_df = pd.concat([S1L,S1B,S1R,S1T,S2L,S2B,S2R,S2T,S3L,S3B,S3R,S3T,S4L,S4B,S4R,S4T])
g4_df = g4_df.reset_index().drop('index', axis=1)

In [12]:
rel_ary = np.array(g4_df['Relative Anode Sensitivity'])
#os.chdir('C:\\Users\\dziubins\\Home\\Research\\Energy Loss Optical Scintillation System\\Data')
file_name = "fine_59Ni_input.dat"

with open(file_name, "w") as file:
    for row in range(len(phodf)):
        rowf = np.array(phodf.loc[row])*rel_ary
        for col in range(120):
            file.write(str(round(rowf[col])))
            file.write(" ")
        file.write("\n")
    
file.close()

In [45]:
rel_ary = np.array(g4_df['Relative Anode Sensitivity'])
df = pd.DataFrame()
rows = []
for row in range(len(phodf)):
    rows.append(np.array(phodf.loc[row])*rel_ary)

In [47]:
df = pd.DataFrame(rows)

In [65]:
difs1 = []
difs2 = []
for row in range(len(phodf)):
    difs1.append(df.loc[row].sum())
    difs2.append(phodf.loc[row].sum())

In [69]:
np.min(difs1), np.min(difs2)


(112853.91949606776, 112866)

In [4]:
phodf = np.loadtxt("PMT Sensitivity with G4\\r8520_59Ni.photon")

In [5]:
phodf

array([[ 624.,  626.,  479., ...,  323.,  123.,   74.],
       [ 916.,  792.,  642., ..., 1541.,  335.,  137.],
       [  23.,   29.,   46., ...,  235.,  225.,  234.],
       ...,
       [ 324.,  464.,  636., ...,  706.,  568.,  412.],
       [  95.,  150.,  279., ...,  417.,  474.,  477.],
       [ 164.,  222.,  376., ...,  392.,  342.,  313.]])